# <u> <font color = "bisque"> Tareas:</font> </u>
## <font color = "bisque"> * Implementar procesos de renombre </font> 

---

<b> <font color = "LightSalmon"> MODO DE TRABAJO PROCESO DE RENOMBRE - v2 </font> </b>

Voy a trabajar sobre argendata/data que esté hasta ahora, en un fork propio.

<b> Sobre argendata/data lo que voy a hacer es tomar los .csv  originales y hacer lo siguiente (dependiendo el caso), generando .csv actualizados:</b>


   - Si un dataset tiene un código que esté incluido en los geocodigos del geonomenclador, se nomencla con la string de name_long

   - Si un dataset tiene un código que NO este incluido en los geocodigos y tiene una string que matchee sin ambigüedad en desc_fundar, entonces se usa el código que está en el geonomenclador (y se guarda la asociación del codigo encontrado con el codigo usado)

   - Si un dataset tiene un código que NO este incluido en los geocodigos y tiene una string que NO matchea sin ambigüedad en desc_fundar, se genera un código nuevo (a revisión para ser desambiguado con posibles matches)

   - Si un dataset tiene una string que matchee sin ambigüedad en desc_fundar entonces se usa el código que está en el geonomenclador para esa string

   - Si un dataset tiene una string que NO matchee sin ambigüedad en desc_fundar, entonces se genera un código nuevo (a revisión para ser desambiguado con posibles matches)


<b> ¿Cómo busco las columnas de interés para esta iteración? </b>
- X% (por ejemplo 80%) del contenido de columnas de interés de un .csv de un tópico es parte de alguna de las columnas del geoneomenclador 
- Por el momento solo código que esté en el geonomenclador en español

Observaciones: 
- PARA ESTE LABURO ARMÉ MI PROPIO BLOQUE DE ENCODERs, DELIMITERs
- Para esta versión de la tarea, en principio, solo consideraría contenido en español

<b> <font color = "LightSalmon"> Productos: 
- .csv argendata con agregado de columnas para implementar renombres correspondientes
- .csv con lo que mencioné antes con .csv de argendata que presentarían problemas </font> </b>


---

###  <u> <font color = "orangered"> Cómo abrir cada .csv considerando su encoding: </font> </u>

<font color = "gold"> Cada .csv de Argendata tiene su propio encodings y delimiters </font>. 

No todos los .csv que contempla Argendata tienen el mismo enconding y delimiter. Por eso antes de comenzar a trabajar voy a armar el siguiente input para la tarea: dataframe que contenga 4 columnas que sean 'TOPICO', 'archivo_csv', 'encoding' y delimiter'.

In [1]:
import os
import glob
import chardet
import csv
import pandas as pd

# ruta de trabajo donde estan los topicos con sus csv 
base_dir =  '/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata' # '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata' #

# lista para recolectar los datos
data = []

# tamanio del fragmento a leer (en este caso 100000 bytes)
sample_size = 100000

for topico in os.listdir(base_dir): # ejemplo SEBACO
    topico_path = os.path.join(base_dir, topico) #ejemplo /home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata/SEBACO

    # ignorar archivos sueltos como LICENSE y README.md
    if not os.path.isdir(topico_path):
        continue

    for archivo in os.listdir(topico_path):
        if archivo.endswith('.csv'):
            archivo_path = os.path.join(topico_path, archivo)

            # leer un fragmento del .csv para análisis
            with open(archivo_path, 'rb') as f:
                raw_sample = f.read(sample_size)

            # detectar encoding
            encoding_detected = 'utf-8'  # por defecto
            try:
                encoding_info = chardet.detect(raw_sample)
                if encoding_info and encoding_info['encoding']:
                    encoding_detected = encoding_info['encoding']
            except:
                pass

            # decodificar el fragmento
            try:
                sample = raw_sample.decode(encoding_detected, errors = 'replace')
            except:
                sample = raw_sample.decode('utf-8', errors = 'replace') 

            # detectar delimitador
            delimiter_detected = ','
            try:
                sniffer = csv.Sniffer()
                dialect = sniffer.sniff(sample)
                delimiter_detected = dialect.delimiter
            except:
                pass

            # agregar al resultado
            data.append({
                'TOPICO': topico,
                'archivo_csv': archivo,
                'encoding': encoding_detected,
                'delimiter': delimiter_detected
            })


df_resultado = pd.DataFrame(data)
df_codificacion = df_resultado

print(df_codificacion)

# guardar como csv el archivo csvTopicoArgendata_encoding_and_delimiters
# path_df_encoding_delimiters = '/home/lucia/Desktop/Fundar/Argendata_renaming/data/'
# df_resultado.to_csv(path_df_encoding_delimiters + 'csvTopicosArgendata_codificacion_y_delimitadores.csv', index = False)


     TOPICO                     archivo_csv    encoding delimiter
0    SEBACO              01_ventas_afip.csv       ascii         ,
1    SEBACO     11_ocupados_x_condicion.csv       utf-8         ,
2    SEBACO   22_participacion_expo_sbc.csv       ascii         ,
3    SEBACO          05_empleo_sbc_oede.csv       ascii         ,
4    SEBACO      16_brecha_salarial_ssi.csv       utf-8         ,
..      ...                             ...         ...       ...
332  MINERI       minerales_vbp_mundial.csv  ISO-8859-1         ,
333  MINERI    importaciones_por_sector.csv       ascii         ,
334  MINERI                  pbi_minero.csv       ascii         ,
335  MINERI     importaciones_por_grupo.csv       ascii         ,
336  MINERI  compras_sector_minero_rama.csv       utf-8         ,

[337 rows x 4 columns]


###  <u> <font color = "orangered">  Tareas concretas del proceso de renaming </font> </u>

<b> <font color = "gold"> Sobre argendata/data lo que voy a hacer es tomar los .csv  originales y hacer lo siguiente (dependiendo el caso), generando .csv actualizados: </font> </b>

   - si un dataset tiene un código que esté incluido en los geocodigos del geonomenclador, se nomencla con la string de name_long (si es que existe), y si no es así se nomencla con la string de desc_fundar correspondiente

   - Si un dataset tiene un código que NO este incluido en los geocodigos y tiene una string que matchee sin ambigüedad en desc_fundar, entonces se usa el código que está en el geonomenclador (y se guarda la asociación del codigo encontrado con el codigo usado)

   - Si un dataset tiene un código que NO este incluido en los geocodigos y tiene una string que NO matchea sin ambigüedad en desc_fundar, se genera un código nuevo (a revisión para ser desambiguado con posibles matches)

   - Si un dataset tiene una string que matchee sin ambigüedad en desc_fundar entonces se usa el código que está en el geonomenclador para esa string

   - Si un dataset tiene una string que NO matchee sin ambigüedad en desc_fundar, entonces se genera un código nuevo (a revisión para ser desambiguado con posibles matches)

<b> <font color = "gold"> Para hacer esto, empecemos por: </font> </b>

1. abriría el archivo según su encoding y delimiter dado por df_codificacion 

2. detectaría si cada .csv de tiene una columna que se corresponda a la columna geocodigo del csv geonomenclador según la siguiente condición: fijarse si, al menos, el 80% de sus filas se encuentra contenida en la columna geocodigo del geonomenclador

3. detectaría si cada .csv de tiene una columna que se corresponda a la columna desc_fundar del csv geonomenclador según la siguiente condición: fijarse si, al menos, el 80% de sus filas se encuentra contenida en la columna desc_fundar del geonomenclador

4. esta información guardarla en un dataframe llamado columnscsv_Geocodigo_descFundar, con formato de 4 columnas que sean 'TOPICO', 'archivo_csv', 'columna_Geocodigo', 'columna_DescFundar'. Donde sea 'columna_Geocodigo' es el nombre de la columna de dicho cvs que cumple la condición 60% (80%, X%) geocodigo del geonomenclador y 'descFundar' es el nombre de la columna de dicho csv que cumple la condición 60% (80%, X%) desc_fundar del geonomenclador. 

A la hora de hacer la comparación del 60%:
- De la columna analizada la columna, eliminar nulos
- normalizar espacios, mayúsculas, tildes, para que casos como los siguientes no rompan el X%: "NEA " y "NEA" o "pampeana" y "Pampeana" o "Córdoba" y "Córdoba" se reconozcan como idénticos

<b> <font color = "bisque"> Esto resulta en un df con: TOPICO | archivo_csv | columna_Geocodigo | columna_DescFundar </font> </b>


In [2]:
import os
import pandas as pd
import unicodedata

# funcion para eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in nkfd if not unicodedata.combining(c))

# rutas y dataframes necesarios

base_dir = '/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata' # '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata' 
# df_codificacion hecho
df_geonomenclador = pd.read_csv('/home/lucia/Desktop/Fundar/Argendata_renaming/data/input' + '/geonomenclador_LuloTest.csv')

# sets normalizados de las columnas de interes del geonomenclador: geocodigo y desc_fundar 
# estoy probndo usar set para practicar, si es necesario paso a lista
set_geocod = set(
    df_geonomenclador['geocodigo']
      .dropna().astype(str) #elimino nulos
      .str.strip() # quitar espacios al inicio/final
      .str.upper() # pasar todo a mayusculas
      .apply(sin_tildes) # sacar tildes
)
set_desc = set(
    df_geonomenclador['desc_fundar']
      .dropna().astype(str)
      .str.strip()
      .str.upper()
      .apply(sin_tildes)
)

results = []

# los topicos (directorios) ignorando los archivos LICENSE y README.md
topics = []
for d in os.listdir(base_dir):
    item_path = os.path.join(base_dir, d)
    
    if os.path.isdir(item_path): # verifico que sea un directorio
        
        if d != 'LICENSE' and d != 'README.md': # no incluyo LICENSE y README.md
            topics.append(d)

for topico in topics:
    topic_dir = os.path.join(base_dir, topico)
    # listo los .csv de este tópico
    csv_files = [f for f in os.listdir(topic_dir) if f.endswith('.csv')]
    for archivo in csv_files:
        # uso el encoding y delimiter de df_codificacion
        row = df_codificacion[
            (df_codificacion['TOPICO'] == topico) &
            (df_codificacion['archivo_csv'] == archivo)
        ]
        if row.empty:
            print(f'No hay info de encoding/delimiter para {topico}/{archivo}') # quiero probar no estar pifiando
            continue
        encoding  = row.iloc[0]['encoding']
        delimiter = row.iloc[0]['delimiter']
        path_csv  = os.path.join(topic_dir, archivo)

        # leo csv con el encoding delimiter
        try:
            df_csv = pd.read_csv(path_csv, encoding = encoding, delimiter = delimiter)
        except Exception as e: 
            print(f'Error leyendo {path_csv}: {e}') # imprimo el error
            continue

        col_geocod = None
        col_desc   = None

        # analizar cada columna del .csv
        for col in df_csv.columns:
            serie = (
                df_csv[col]
                  .dropna().astype(str)
                  .str.strip()
                  .str.upper()
                  .apply(sin_tildes)
            )
            n = len(serie)
            if n == 0:
                continue

            match_geo  = serie.isin(set_geocod).sum() / n
            match_desc = serie.isin(set_desc).sum()   / n

            if match_geo  >= 0.6 and col_geocod is None:
                col_geocod = col
            if match_desc >= 0.6 and col_desc   is None:
                col_desc = col

        results.append({
            'TOPICO': topico,
            'archivo_csv': archivo,
            'columna_Geocodigo': col_geocod,
            'columna_DescFundar': col_desc
        })

# creo df de interes
columnscsv_Geocodigo_descFundar = pd.DataFrame(results)

columnscsv_Geocodigo_descFundar

,TOPICO,archivo_csv,columna_Geocodigo,columna_DescFundar
0,SEBACO,01_ventas_afip.csv,None,None
1,SEBACO,11_ocupados_x_condicion.csv,None,None
2,SEBACO,22_participacion_expo_sbc.csv,iso3,None
3,SEBACO,05_empleo_sbc_oede.csv,None,None
4,SEBACO,16_brecha_salarial_ssi.csv,None,None
...,...,...,...,...
332,MINERI,minerales_vbp_mundial.csv,codigo_pais,None
333,MINERI,importaciones_por_sector.csv,None,None
334,MINERI,pbi_minero.csv,None,None
335,MINERI,importaciones_por_grupo.csv,None,None


In [3]:
df_geonomenclador

,geocodigo,desc_fundar,name_long,name_short,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7
0,ABW,Aruba,NaN,NaN,NaN,NaN,NaN,NaN
1,AFE,África Oriental y Meridional,NaN,NaN,NaN,NaN,NaN,NaN
2,AFG,Afganistán,NaN,NaN,NaN,NaN,NaN,NaN
3,AFW,África Occidental y Central,NaN,NaN,NaN,NaN,NaN,NaN
4,AGO,Angola,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
1019,DESHUM_AHDI.WOF,Ramificaciones de Occidente (AHDI),Ramificaciones de Occidente,Ramificaciones de Occidente,NaN,NaN,NaN,NaN
1020,NIR,Irlanda del Norte,NaN,NaN,NaN,NaN,NaN,NaN
1021,CAMCLI_IA,Aviación internacional,NaN,NaN,NaN,NaN,NaN,NaN
1022,F_ESTPRO,África (OECD),África,África,NaN,NaN,NaN,NaN


<b> <font color = "gold"> Continuamos con lo que enumero a continuación para cumplir las condiciones detalladas enteriormente (los 5 bullets de más arriba): </font> </b>

Ya generamos el df columnscsv_Geocodigo_descFundar con: TOPICO | archivo_csv | columna_Geocodigo | columna_DescFundar, 
Donde 'columna_Geocodigo' es el nombre de la columna de dicho cvs que cumple la condición X% geocodigo del geonomenclador y 'descFundar' es el nombre de la columna de dicho csv que cumple la condición X% desc_fundar del geonomenclador.

<b> <font color = "gold">Por cada tópico (carpeta) y cada csv dentro de este, busco generar una nueva carpeta de cada tópico y dentro de cada carpeta guardar una copia de sus respectivos csvs pero agregandoles a los mismos dos columnas: una que sea 'geocodigoFundar' y otra que sea 'geonombreFundar'.</font> </b>

Si para un csv de un topico tanto columna_Geocodigo como columna_DescFundar es NA, ya sabremos que el csv copia se guardará con NA tanto en las columnas 'geocodigoFundar' como 'geonombreFundar'.

Siendo que columnscsv_Geocodigo_descFundar ya tiene por cada tópico y csv, si dicho csv tiene una columna 'columna_Geocodigo' que sería de geocodigo (condicion X% de columna geocodigo del df_geonomenclador) y otra columna 'columna_DescFundar' (condicion del 80 % de la columna desc_fundar del df_geonomenclador), voy a usarcolumnscsv_Geocodigo_descFundar para completar 'geocodigoFundar' y 'geonombreFundar'.

Para rellenar las columnas 'geocodigoFundar' y 'geonombreFundar', hay que tener las siguientes consideraciones:

- Si un dataset (cada csv original de cada topico) tiene una columna asociada a geocodigo segun columnscsv_Geocodigo_descFundar, el contenido de la columna geocodigoFundar del csv copia tendrá el contenido de la columna asociada a geocodigo segun columnscsv_Geocodigo_descFundar. La columna geonombreFundar del csv copia llenará cada una de sus filas acorde al name_long (si es que existe), y si no es así al string de desc_fundar correspondiente a ese geocodigo del geonomenclador.

- Si un dataset tiene una string que matchee sin ambigüedad en desc_fundar del geonomenclador entonces se usa el código que está en el geonomenclador para esa string. Similar al caso anterior.

- En el caso de que un dataset tiene un código que NO esté incluido en los geocodigos pero tiene una string que matchee sin ambigüedad en desc_fundar, entonces se usa el código que está en geocódigo en el geonomenclador. Esto se registrará en df_problemas con las columnas TOPICO | csv | problema: "dataset tiene el codigo [...], que NO esta incluido en los geocodigos pero tiene la string [...] que matchea sin ambiguedad en desc_fundar. No hay que modificar geonomenclador.". Por cada problema se guarda la string y el código correspondiente. Esto puede dar lugar que un mismo csv de un tópico tenga varios problemas a registrar. NO HAY QUE MODIFICAR GEONOMENCLADOR, EL RENAMING ES HECHO A PARTIR DE LA COLUMNA desc_fundar DEL GEONOMENCLADOR. 

- Si un dataset tiene un código que NO este incluido en los geocodigos y tiene una string que NO matchea sin ambigüedad en desc_fundar, se genera un código nuevo. Esto se registrará en df_problemas con las columnas TOPICO | csv | problema: "dataset tiene el codigo [...] que NO esta incluido en los geocodigos y tiene la string [...] que NO matchea sin ambiguedad en desc_fundar. Puede implicar modificaciones del geonomenclador (a revisión, para no pisarse con otros casos).". Por cada problema se guarda la string y el código correspondiente. Esto puede dar lugar que un mismo csv de un tópico tenga varios problemas a registrar. PODRÍA DAR LUGAR A MODIFICACIONES DEL GEONOMENCLADOR. 

    -Situacion similar a esta: Dataset tiene el código [...] que NO está incluido en los geocódigos y no existe columna desc_fundar. En df_problemas se reporta: "dataset tiene el codigo [...] que NO esta incluido en los geocodigos y no existe columna desc_fundar. Puede implicar modificaciones del geonomenclador (a revisión, para no pisarse con otros casos)."

- Si un dataset tiene una string que NO matchee sin ambigüedad en desc_fundar, entonces guardar este caso en el dataframe llamado df_problemas con las columnas TOPICO | csv | problema, donde TOPICO es el topico con el que se corresponde, csv el csv evaluado y en problema poner: "Para la string [...] se debería generar un codigo nuevo (a revisión, para no pisarse con otros casos)". PODRÍA DAR LUGAR A MODIFICACIONES DEL GEONOMENCLADOR. 

<b> <font color = "bisque"> VAMOS CON ALGUNOS EJEMPLOS PARTICULARES </font> </b>

In [4]:
# PRUEBA SOBRE UN CSV DE UN TOPICO ARGENDATA QUE ESPERO QUE NO TENGA PROBLEMAS: SE GENEROOOOO!!!! :D

import os
import pandas as pd
import unicodedata

# funcion eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in nkfd if not unicodedata.combining(c))

# parametros: especificar el tópico y el csv de ese topico
TOPICO    = 'AGROPE'  
ARCHIVO   = 'produccion_global_carne_aviar_2021.csv'

# rutas
base_dir = '/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata'
dest_dir = '/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata_new'

# df inputs
df_cod = df_codificacion
df_geo = df_geonomenclador
df_cols = columnscsv_Geocodigo_descFundar  

# dicc de name_long y desc_fundar por geocodigo del geonomenclador argendata
geonames_by_geocode = {}
for _, row in df_geo.iterrows():
    code = sin_tildes(str(row['geocodigo']).strip().upper())
    geonames_by_geocode[code] = (row['name_long'], row['desc_fundar'])

# abro el csv con respectivos encoding y delimiter, cfg de configuracion 
cfg = df_cod[(df_cod['TOPICO']==TOPICO) & 
             (df_cod['archivo_csv']==ARCHIVO)]
if cfg.empty:
    print(f'No se encontro info de encoding/delimiter para {TOPICO}/{ARCHIVO}')
encoding   = cfg.iloc[0]['encoding']
delimiter = cfg.iloc[0]['delimiter']

src_path = os.path.join(base_dir, TOPICO, ARCHIVO)
df = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) #csv de interes como un dartaframe
df_original = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) 

# print(df)

# le sumo al csv las nuevas columnas de interes
df['geocodigoFundar'] = pd.NA
df['geonombreFundar'] = pd.NA

print(df)

# del df columnscsv_Geocodigo_descFundar toma la info del csv y topico que me interesa
row_cols = df_cols[(df_cols['TOPICO'] == TOPICO) & (df_cols['archivo_csv'] == ARCHIVO)]
col_code = row_cols.iloc[0]['columna_Geocodigo']   if not row_cols.empty else None
col_desc = row_cols.iloc[0]['columna_DescFundar']  if not row_cols.empty else None

print(col_code)

# lista para tomar nota de los "problemas"
problems = []

# recorro cada fila del 
for idx in df.index:
    # CASO A: existe columna correspondiente a geocodigo del geonomenclador en el csv

    # A.0) hay un codigo presente en geocodigos, lo relleno con name_long o desc_fundar
    if pd.notna(col_code):
        raw_code = str(df.at[idx, col_code])
        norm_code = sin_tildes(raw_code.strip().upper())
        if norm_code in geonames_by_geocode:
            nl, desc0 = geonames_by_geocode[norm_code]
            df.at[idx, 'geocodigoFundar'] = norm_code
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0 # relleno con name_long si existe, sino relleno con la sting de desc_fundar
        else:
            # A.1) hay un codigo NO presente en geocodigos, pero intento por columna desc_fundar
            if pd.notna(col_desc):
                raw_desc = str(df.at[idx, col_desc])
                norm_desc = sin_tildes(raw_desc.strip().upper()) # normalizo columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%
                # bsqueda de coincidencias en desc_fundar
                matched_code = []
                for code_key, (_nl, desc0) in geonames_by_geocode.items(): # _nl para hacer explicito que no uso nl en este caso
                    norm_desc0 = sin_tildes(str(desc0).strip().upper()) # normalizo desc_fundar del geonomenclador 
                    #(para comparacion con la columna del csv de interes columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%)
                    if norm_desc0 == norm_desc:
                        matched_code.append(code_key)
                if len(matched_code) == 1:
                    c0 = matched_code[0]
                    nl, desc0 = geonames_by_geocode[c0]
                    df.at[idx, 'geocodigoFundar'] = c0
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}', que NO esta incluido en los geocodigos pero tiene la string '{norm_desc}' que matchea sin ambiguedad en desc_fundar."
                                     })
                else:
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y tiene la string '{norm_desc}' que NO matchea sin ambiguedad en desc_fundar. Se debe generar un código nuevo (a revisión para ser desambiguado con posibles matches)."
                                    })
                    
            # A.2) no hay columna desc_fundar para intentar
            else:
                problems.append({
                    'TOPICO': TOPICO,
                    'csv': ARCHIVO,
                    'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y no existe columna desc_fundar. A revisar qué se hace." 
                                })

    # Caso B: no existe columna de geocódigo, pero si se tiene de desc_fundar
    elif pd.notna(col_desc):
        raw_desc = str(df.at[idx, col_desc])
        norm_desc = sin_tildes(raw_desc.strip().upper())
        # busqueda coincidencias en desc_fundar
        matched_code = []
        for code_key, (_nl, desc0) in geonames_by_geocode.items():
            norm_desc0 = sin_tildes(str(desc0).strip().upper())
            if norm_desc0 == norm_desc:
                matched_code.append(code_key)
        if len(matched_code) == 1:
            c0 = matched_code[0]
            nl, desc0 = geonames_by_geocode[c0]
            df.at[idx, 'geocodigoFundar'] = c0
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
        else:
            problems.append({
                'TOPICO': TOPICO,
                'csv': ARCHIVO,
                'problema': f"Para la string '{norm_desc}' generar un codigo nuevo (a revison para ser desambiguado con posibles matches)"
                            })

    # Caso C: no hay columnas correspondientes a geocodigo ni de desc_fundar, entonces dejamos NaN

# gurdo csv modificado
os.makedirs(os.path.join(dest_dir, TOPICO), exist_ok = True)
dst_path = os.path.join(dest_dir, TOPICO, ARCHIVO)
df.to_csv(dst_path, index = False)
print(df)

# genero dataframe de problemas
df_problemas = pd.DataFrame(problems)
print(df_problemas)



    iso3        iso3_desc_fundar       valor geocodigoFundar geonombreFundar
0    AFG              Afganistán    28480.02            <NA>            <NA>
1    AGO                  Angola    55384.74            <NA>            <NA>
2    ALB                 Albania    13250.00            <NA>            <NA>
3    ARE  Emiratos Árabes Unidos    46494.00            <NA>            <NA>
4    ARG               Argentina  2294101.41            <NA>            <NA>
..   ...                     ...         ...             ...             ...
190  WSM                   Samoa      403.33            <NA>            <NA>
191  YEM                   Yemen   198878.58            <NA>            <NA>
192  ZAF               Sudáfrica  1910000.00            <NA>            <NA>
193  ZMB                  Zambia    50533.34            <NA>            <NA>
194  ZWE                Zimbabwe   113000.00            <NA>            <NA>

[195 rows x 5 columns]
iso3
    iso3        iso3_desc_fundar       valor ge

In [5]:
# PRUEBA SOBRE OTRO CSV DE UN TOPICO ARGENDATA CON PROBLEMA: 
# TOPICO    = 'MERTRA'  
# ARCHIVO   = 'lavarropas_tasa_empleo_fem_provincia.csv'
# Para la string 'CABA' generar un codigo nuevo (a revison para ser desambiguado con posibles matches)


import os
import pandas as pd
import unicodedata

# funcion eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in nkfd if not unicodedata.combining(c))

# parametros: especificar el tópico y el csv de ese topico
TOPICO    = 'MERTRA'  
ARCHIVO   = 'lavarropas_tasa_empleo_fem_provincia.csv'

# rutas
base_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata'
dest_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata_new'

# df inputs
df_cod = df_codificacion
df_geo = df_geonomenclador
df_cols = columnscsv_Geocodigo_descFundar  

# dicc de name_long y desc_fundar por geocodigo del geonomenclador argendata
geonames_by_geocode = {}
for _, row in df_geo.iterrows():
    code = sin_tildes(str(row['geocodigo']).strip().upper())
    geonames_by_geocode[code] = (row['name_long'], row['desc_fundar'])

# abro el csv con respectivos encoding y delimiter, cfg de configuracion 
cfg = df_cod[(df_cod['TOPICO']==TOPICO) & 
             (df_cod['archivo_csv']==ARCHIVO)]
if cfg.empty:
    print(f'No se encontro info de encoding/delimiter para {TOPICO}/{ARCHIVO}')
encoding   = cfg.iloc[0]['encoding']
delimiter = cfg.iloc[0]['delimiter']

src_path = os.path.join(base_dir, TOPICO, ARCHIVO)
df = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) #csv de interes como un dartaframe
df_original = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) 

# print(df)

# le sumo al csv las nuevas columnas de interes
df['geocodigoFundar'] = pd.NA
df['geonombreFundar'] = pd.NA

print(df)

# del df columnscsv_Geocodigo_descFundar toma la info del csv y topico que me interesa
row_cols = df_cols[(df_cols['TOPICO'] == TOPICO) & (df_cols['archivo_csv'] == ARCHIVO)]
col_code = row_cols.iloc[0]['columna_Geocodigo']   if not row_cols.empty else None
col_desc = row_cols.iloc[0]['columna_DescFundar']  if not row_cols.empty else None

print(col_code)

# lista para tomar nota de los "problemas"
problems = []

# recorro cada fila del 
for idx in df.index:
    # CASO A: existe columna correspondiente a geocodigo del geonomenclador en el csv

    # A.0) hay un codigo presente en geocodigos, lo relleno con name_long o desc_fundar
    if pd.notna(col_code):
        raw_code = str(df.at[idx, col_code])
        norm_code = sin_tildes(raw_code.strip().upper())
        if norm_code in geonames_by_geocode:
            nl, desc0 = geonames_by_geocode[norm_code]
            df.at[idx, 'geocodigoFundar'] = norm_code
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0 # relleno con name_long si existe, sino relleno con la sting de desc_fundar
        else:
            # A.1) hay un codigo NO presente en geocodigos, pero intento por columna desc_fundar
            if pd.notna(col_desc):
                raw_desc = str(df.at[idx, col_desc])
                norm_desc = sin_tildes(raw_desc.strip().upper()) # normalizo columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%
                # bsqueda de coincidencias en desc_fundar
                matched_code = []
                for code_key, (_nl, desc0) in geonames_by_geocode.items(): # _nl para hacer explicito que no uso nl en este caso
                    norm_desc0 = sin_tildes(str(desc0).strip().upper()) # normalizo desc_fundar del geonomenclador 
                    #(para comparacion con la columna del csv de interes columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%)
                    if norm_desc0 == norm_desc:
                        matched_code.append(code_key)
                if len(matched_code) == 1:
                    c0 = matched_code[0]
                    nl, desc0 = geonames_by_geocode[c0]
                    df.at[idx, 'geocodigoFundar'] = c0
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}', que NO esta incluido en los geocodigos pero tiene la string '{norm_desc}' que matchea sin ambiguedad en desc_fundar."
                                     })
                else:
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y tiene la string '{norm_desc}' que NO matchea sin ambiguedad en desc_fundar. Se debe generar un código nuevo (a revisión para ser desambiguado con posibles matches)."
                                    })
                    
            # A.2) no hay columna desc_fundar para intentar
            else:
                problems.append({
                    'TOPICO': TOPICO,
                    'csv': ARCHIVO,
                    'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y no existe columna desc_fundar. A revisar qué se hace." 
                                })

    # CASO B: no existe columna de geocodigo, pero si se tiene de desc_fundar

    elif pd.notna(col_desc):
        raw_desc = str(df.at[idx, col_desc])
        norm_desc = sin_tildes(raw_desc.strip().upper())
        # busqueda coincidencias en desc_fundar
        matched_code = []
        for code_key, (_nl, desc0) in geonames_by_geocode.items():
            norm_desc0 = sin_tildes(str(desc0).strip().upper())
            if norm_desc0 == norm_desc:
                matched_code.append(code_key)
        if len(matched_code) == 1:
            c0 = matched_code[0]
            nl, desc0 = geonames_by_geocode[c0]
            df.at[idx, 'geocodigoFundar'] = c0
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
        else:
            problems.append({
                'TOPICO': TOPICO,
                'csv': ARCHIVO,
                'problema': f"Para la string '{norm_desc}' generar un codigo nuevo (a revison para ser desambiguado con posibles matches)"
                            })

    # CASO C: no hay columnas correspondientes a geocodigo ni de desc_fundar, queda NaN

# gurdo csv modificado
os.makedirs(os.path.join(dest_dir, TOPICO), exist_ok = True)
dst_path = os.path.join(dest_dir, TOPICO, ARCHIVO)
df.to_csv(dst_path, index = False)
print(df)

# genero dataframe de problemas
df_problemas = pd.DataFrame(problems)
print(df_problemas)



    prov_cod            prov_desc  tasa_empleo_18_65_mujeres  \
0          2                 CABA                   0.755180   
1          6         Buenos Aires                   0.625285   
2         10            Catamarca                   0.590940   
3         14              Córdoba                   0.585850   
4         18           Corrientes                   0.482550   
5         22                Chaco                   0.505072   
6         26               Chubut                   0.594072   
7         30           Entre Ríos                   0.569709   
8         34              Formosa                   0.421378   
9         38                Jujuy                   0.623202   
10        42             La Pampa                   0.623782   
11        46             La Rioja                   0.585750   
12        50              Mendoza                   0.588962   
13        54             Misiones                   0.611542   
14        58              Neuquén       

In [6]:
# PRUEBA SOBRE OTRO CSV DE UN TOPICO ARGENDATA CON PROBLEMA: 
# TOPICO    = 'MERTRA'  
# ARCHIVO   = 'ratio_tasa_actividad_mujer_varon_por_pais_anio.csv'
# ya no tiene problemas :D

import os
import pandas as pd
import unicodedata

# funcion eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in nkfd if not unicodedata.combining(c))

# parametros: especificar el tópico y el csv de ese topico
TOPICO    = 'MERTRA'  
ARCHIVO   = 'ratio_tasa_actividad_mujer_varon_por_pais_anio.csv'

# rutas
base_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata'
dest_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata_new'

# df inputs
df_cod = df_codificacion
df_geo = df_geonomenclador
df_cols = columnscsv_Geocodigo_descFundar  

# dicc de name_long y desc_fundar por geocodigo del geonomenclador argendata
geonames_by_geocode = {}
for _, row in df_geo.iterrows():
    code = sin_tildes(str(row['geocodigo']).strip().upper())
    geonames_by_geocode[code] = (row['name_long'], row['desc_fundar'])

# abro el csv con respectivos encoding y delimiter, cfg de configuracion 
cfg = df_cod[(df_cod['TOPICO']==TOPICO) & 
             (df_cod['archivo_csv']==ARCHIVO)]
if cfg.empty:
    print(f'No se encontro info de encoding/delimiter para {TOPICO}/{ARCHIVO}')
encoding   = cfg.iloc[0]['encoding']
delimiter = cfg.iloc[0]['delimiter']

src_path = os.path.join(base_dir, TOPICO, ARCHIVO)
df = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) #csv de interes como un dartaframe
df_original = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) 

# print(df)

# le sumo al csv las nuevas columnas de interes
df['geocodigoFundar'] = pd.NA
df['geonombreFundar'] = pd.NA

print(df)

# del df columnscsv_Geocodigo_descFundar toma la info del csv y topico que me interesa
row_cols = df_cols[(df_cols['TOPICO'] == TOPICO) & (df_cols['archivo_csv'] == ARCHIVO)]
col_code = row_cols.iloc[0]['columna_Geocodigo']   if not row_cols.empty else None
col_desc = row_cols.iloc[0]['columna_DescFundar']  if not row_cols.empty else None

print(col_code)

# lista para tomar nota de los "problemas"
problems = []

# recorro cada fila del 
for idx in df.index:
    # CASO A: existe columna correspondiente a geocodigo del geonomenclador en el csv

    # A.0) hay un codigo presente en geocodigos, lo relleno con name_long o desc_fundar
    if pd.notna(col_code):
        raw_code = str(df.at[idx, col_code])
        norm_code = sin_tildes(raw_code.strip().upper())
        if norm_code in geonames_by_geocode:
            nl, desc0 = geonames_by_geocode[norm_code]
            df.at[idx, 'geocodigoFundar'] = norm_code
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0 # relleno con name_long si existe, sino relleno con la sting de desc_fundar
        else:
            # A.1) hay un codigo NO presente en geocodigos, pero intento por columna desc_fundar
            if pd.notna(col_desc):
                raw_desc = str(df.at[idx, col_desc])
                norm_desc = sin_tildes(raw_desc.strip().upper()) # normalizo columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%
                # bsqueda de coincidencias en desc_fundar
                matched_code = []
                for code_key, (_nl, desc0) in geonames_by_geocode.items(): # _nl para hacer explicito que no uso nl en este caso
                    norm_desc0 = sin_tildes(str(desc0).strip().upper()) # normalizo desc_fundar del geonomenclador 
                    #(para comparacion con la columna del csv de interes columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%)
                    if norm_desc0 == norm_desc:
                        matched_code.append(code_key)
                if len(matched_code) == 1:
                    c0 = matched_code[0]
                    nl, desc0 = geonames_by_geocode[c0]
                    df.at[idx, 'geocodigoFundar'] = c0
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}', que NO esta incluido en los geocodigos pero tiene la string '{norm_desc}' que matchea sin ambiguedad en desc_fundar."
                                     })
                else:
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y tiene la string '{norm_desc}' que NO matchea sin ambiguedad en desc_fundar. Se debe generar un código nuevo (a revisión para ser desambiguado con posibles matches)."
                                    })
                    
            # A.2) no hay columna desc_fundar para intentar
            else:
                problems.append({
                    'TOPICO': TOPICO,
                    'csv': ARCHIVO,
                    'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y no existe columna desc_fundar. A revisar qué se hace." 
                                })

    # Caso B: no existe columna de geocódigo, pero si se tiene de desc_fundar
    elif pd.notna(col_desc):
        raw_desc = str(df.at[idx, col_desc])
        norm_desc = sin_tildes(raw_desc.strip().upper())
        # busqueda coincidencias en desc_fundar
        matched_code = []
        for code_key, (_nl, desc0) in geonames_by_geocode.items():
            norm_desc0 = sin_tildes(str(desc0).strip().upper())
            if norm_desc0 == norm_desc:
                matched_code.append(code_key)
        if len(matched_code) == 1:
            c0 = matched_code[0]
            nl, desc0 = geonames_by_geocode[c0]
            df.at[idx, 'geocodigoFundar'] = c0
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
        else:
            problems.append({
                'TOPICO': TOPICO,
                'csv': ARCHIVO,
                'problema': f"Para la string '{norm_desc}' generar un codigo nuevo (a revison para ser desambiguado con posibles matches)"
                            })

    # Caso C: no hay columnas correspondientes a geocodigo ni de desc_fundar, entonces dejamos NaN

# gurdo csv modificado
os.makedirs(os.path.join(dest_dir, TOPICO), exist_ok = True)
dst_path = os.path.join(dest_dir, TOPICO, ARCHIVO)
df.to_csv(dst_path, index = False)
print(df)

# genero dataframe de problemas
df_problemas = pd.DataFrame(problems)
print(df_problemas)



      iso3                    iso3_desc  anio  \
0      AFE  Africa Eastern and Southern  2023   
1      AFE  Africa Eastern and Southern  2022   
2      AFE  Africa Eastern and Southern  2021   
3      AFE  Africa Eastern and Southern  2020   
4      AFE  Africa Eastern and Southern  2019   
...    ...                          ...   ...   
16699  ZWE                     Zimbabwe  1964   
16700  ZWE                     Zimbabwe  1963   
16701  ZWE                     Zimbabwe  1962   
16702  ZWE                     Zimbabwe  1961   
16703  ZWE                     Zimbabwe  1960   

       ratio_tasa_actividad_mujer_varon nivel_agregacion geocodigoFundar  \
0                                   NaN       agregacion            <NA>   
1                                   NaN       agregacion            <NA>   
2                                   NaN       agregacion            <NA>   
3                                   NaN       agregacion            <NA>   
4                              

In [7]:
# PRUEBA SOBRE OTRO CSV DE UN TOPICO ARGENDATA CON PROBLEMA: 
# TOPICO    = 'ACECON'  
# ARCHIVO   = '9_pibpc_ppa_log_1950.csv'
# Para la string 'TIGRES ASIATICOS' generar un codigo nuevo (a revison para ser desambiguado con posibles matches)

import os
import pandas as pd
import unicodedata

# funcion eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in nkfd if not unicodedata.combining(c))

# parametros: especificar el tópico y el csv de ese topico
TOPICO    = 'ACECON'  
ARCHIVO   = '9_pibpc_ppa_log_1950.csv'

# rutas
base_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata'
dest_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata_new'

# df inputs
df_cod = df_codificacion
df_geo = df_geonomenclador
df_cols = columnscsv_Geocodigo_descFundar  

# dicc de name_long y desc_fundar por geocodigo del geonomenclador argendata
geonames_by_geocode = {}
for _, row in df_geo.iterrows():
    code = sin_tildes(str(row['geocodigo']).strip().upper())
    geonames_by_geocode[code] = (row['name_long'], row['desc_fundar'])

# abro el csv con respectivos encoding y delimiter, cfg de configuracion 
cfg = df_cod[(df_cod['TOPICO']==TOPICO) & 
             (df_cod['archivo_csv']==ARCHIVO)]
if cfg.empty:
    print(f'No se encontro info de encoding/delimiter para {TOPICO}/{ARCHIVO}')
encoding   = cfg.iloc[0]['encoding']
delimiter = cfg.iloc[0]['delimiter']

src_path = os.path.join(base_dir, TOPICO, ARCHIVO)
df = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) #csv de interes como un dartaframe
df_original = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) 

# print(df)

# le sumo al csv las nuevas columnas de interes
df['geocodigoFundar'] = pd.NA
df['geonombreFundar'] = pd.NA

print(df)

# del df columnscsv_Geocodigo_descFundar toma la info del csv y topico que me interesa
row_cols = df_cols[(df_cols['TOPICO'] == TOPICO) & (df_cols['archivo_csv'] == ARCHIVO)]
col_code = row_cols.iloc[0]['columna_Geocodigo']   if not row_cols.empty else None
col_desc = row_cols.iloc[0]['columna_DescFundar']  if not row_cols.empty else None

print(col_code)

# lista para tomar nota de los "problemas"
problems = []

# recorro cada fila del 
for idx in df.index:
    # CASO A: existe columna correspondiente a geocodigo del geonomenclador en el csv

    # A.0) hay un codigo presente en geocodigos, lo relleno con name_long o desc_fundar
    if pd.notna(col_code):
        raw_code = str(df.at[idx, col_code])
        norm_code = sin_tildes(raw_code.strip().upper())
        if norm_code in geonames_by_geocode:
            nl, desc0 = geonames_by_geocode[norm_code]
            df.at[idx, 'geocodigoFundar'] = norm_code
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0 # relleno con name_long si existe, sino relleno con la sting de desc_fundar
        else:
            # A.1) hay un codigo NO presente en geocodigos, pero intento por columna desc_fundar
            if pd.notna(col_desc):
                raw_desc = str(df.at[idx, col_desc])
                norm_desc = sin_tildes(raw_desc.strip().upper()) # normalizo columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%
                # bsqueda de coincidencias en desc_fundar
                matched_code = []
                for code_key, (_nl, desc0) in geonames_by_geocode.items(): # _nl para hacer explicito que no uso nl en este caso
                    norm_desc0 = sin_tildes(str(desc0).strip().upper()) # normalizo desc_fundar del geonomenclador 
                    #(para comparacion con la columna del csv de interes columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%)
                    if norm_desc0 == norm_desc:
                        matched_code.append(code_key)
                if len(matched_code) == 1:
                    c0 = matched_code[0]
                    nl, desc0 = geonames_by_geocode[c0]
                    df.at[idx, 'geocodigoFundar'] = c0
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}', que NO esta incluido en los geocodigos pero tiene la string '{norm_desc}' que matchea sin ambiguedad en desc_fundar."
                                     })
                else:
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y tiene la string '{norm_desc}' que NO matchea sin ambiguedad en desc_fundar. Se debe generar un código nuevo (a revisión para ser desambiguado con posibles matches)."
                                    })
                    
            # A.2) no hay columna desc_fundar para intentar
            else:
                problems.append({
                    'TOPICO': TOPICO,
                    'csv': ARCHIVO,
                    'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y no existe columna desc_fundar. A revisar qué se hace." 
                                })

    # Caso B: no existe columna de geocódigo, pero si se tiene de desc_fundar
    elif pd.notna(col_desc):
        raw_desc = str(df.at[idx, col_desc])
        norm_desc = sin_tildes(raw_desc.strip().upper())
        # busqueda coincidencias en desc_fundar
        matched_code = []
        for code_key, (_nl, desc0) in geonames_by_geocode.items():
            norm_desc0 = sin_tildes(str(desc0).strip().upper())
            if norm_desc0 == norm_desc:
                matched_code.append(code_key)
        if len(matched_code) == 1:
            c0 = matched_code[0]
            nl, desc0 = geonames_by_geocode[c0]
            df.at[idx, 'geocodigoFundar'] = c0
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
        else:
            problems.append({
                'TOPICO': TOPICO,
                'csv': ARCHIVO,
                'problema': f"Para la string '{norm_desc}' generar un codigo nuevo (a revison para ser desambiguado con posibles matches)"
                            })

    # Caso C: no hay columnas correspondientes a geocodigo ni de desc_fundar, entonces dejamos NaN

# gurdo csv modificado
os.makedirs(os.path.join(dest_dir, TOPICO), exist_ok = True)
dst_path = os.path.join(dest_dir, TOPICO, ARCHIVO)
df.to_csv(dst_path, index = False)
print(df)

# genero dataframe de problemas
df_problemas = pd.DataFrame(problems)
print(df_problemas)



     anio     pais_o_region  pbi_per_capita_ppa geocodigoFundar  \
0    1950    America Latina         3672.315179            <NA>   
1    1951    America Latina         3772.801926            <NA>   
2    1952    America Latina         3800.175320            <NA>   
3    1953    America Latina         3852.794160            <NA>   
4    1954    America Latina         3973.484863            <NA>   
..    ...               ...                 ...             ...   
214  2018  Tigres Asiaticos        42850.848420            <NA>   
215  2019  Tigres Asiaticos        43538.234690            <NA>   
216  2020  Tigres Asiaticos        43324.091790            <NA>   
217  2021  Tigres Asiaticos        46071.454090            <NA>   
218  2022  Tigres Asiaticos        47014.943250            <NA>   

    geonombreFundar  
0              <NA>  
1              <NA>  
2              <NA>  
3              <NA>  
4              <NA>  
..              ...  
214            <NA>  
215            <NA>

In [8]:
# PRUEBA SOBRE OTRO CSV DE UN TOPICO ARGENDATA CON PROBLEMA: 
# TOPICO    = 'POBREZ'  
# ARCHIVO   = 'ISA_global-poverty_it3.csv'
# no se registra ni columna similar a geocodigo ni desc_fundar, a pesar de tener una columna llamada region porque el contenido es ‘GBA' y 'national' 
# que son georreferencial pero no son parte de desc_fundar, y no tiene una columna de geocodigo tampoco

import os
import pandas as pd
import unicodedata

# funcion eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in nkfd if not unicodedata.combining(c))

# parametros: especificar el tópico y el csv de ese topico
TOPICO    = 'POBREZ'  
ARCHIVO   = 'ISA_global-poverty_it3.csv'

# rutas
base_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata'
dest_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata_new'

# df inputs
df_cod = df_codificacion
df_geo = df_geonomenclador
df_cols = columnscsv_Geocodigo_descFundar  

# dicc de name_long y desc_fundar por geocodigo del geonomenclador argendata
geonames_by_geocode = {}
for _, row in df_geo.iterrows():
    code = sin_tildes(str(row['geocodigo']).strip().upper())
    geonames_by_geocode[code] = (row['name_long'], row['desc_fundar'])

# abro el csv con respectivos encoding y delimiter, cfg de configuracion 
cfg = df_cod[(df_cod['TOPICO']==TOPICO) & 
             (df_cod['archivo_csv']==ARCHIVO)]
if cfg.empty:
    print(f'No se encontro info de encoding/delimiter para {TOPICO}/{ARCHIVO}')
encoding   = cfg.iloc[0]['encoding']
delimiter = cfg.iloc[0]['delimiter']

src_path = os.path.join(base_dir, TOPICO, ARCHIVO)
df = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) #csv de interes como un dartaframe
df_original = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) 

# print(df)

# le sumo al csv las nuevas columnas de interes
df['geocodigoFundar'] = pd.NA
df['geonombreFundar'] = pd.NA

print(df)

# del df columnscsv_Geocodigo_descFundar toma la info del csv y topico que me interesa
row_cols = df_cols[(df_cols['TOPICO'] == TOPICO) & (df_cols['archivo_csv'] == ARCHIVO)]
col_code = row_cols.iloc[0]['columna_Geocodigo']   if not row_cols.empty else None
col_desc = row_cols.iloc[0]['columna_DescFundar']  if not row_cols.empty else None

print(col_code)

# lista para tomar nota de los "problemas"
problems = []

# recorro cada fila del 
for idx in df.index:
    # CASO A: existe columna correspondiente a geocodigo del geonomenclador en el csv

    # A.0) hay un codigo presente en geocodigos, lo relleno con name_long o desc_fundar
    if pd.notna(col_code):
        raw_code = str(df.at[idx, col_code])
        norm_code = sin_tildes(raw_code.strip().upper())
        if norm_code in geonames_by_geocode:
            nl, desc0 = geonames_by_geocode[norm_code]
            df.at[idx, 'geocodigoFundar'] = norm_code
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0 # relleno con name_long si existe, sino relleno con la sting de desc_fundar
        else:
            # A.1) hay un codigo NO presente en geocodigos, pero intento por columna desc_fundar
            if pd.notna(col_desc):
                raw_desc = str(df.at[idx, col_desc])
                norm_desc = sin_tildes(raw_desc.strip().upper()) # normalizo columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%
                # bsqueda de coincidencias en desc_fundar
                matched_code = []
                for code_key, (_nl, desc0) in geonames_by_geocode.items(): # _nl para hacer explicito que no uso nl en este caso
                    norm_desc0 = sin_tildes(str(desc0).strip().upper()) # normalizo desc_fundar del geonomenclador 
                    #(para comparacion con la columna del csv de interes columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%)
                    if norm_desc0 == norm_desc:
                        matched_code.append(code_key)
                if len(matched_code) == 1:
                    c0 = matched_code[0]
                    nl, desc0 = geonames_by_geocode[c0]
                    df.at[idx, 'geocodigoFundar'] = c0
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}', que NO esta incluido en los geocodigos pero tiene la string '{norm_desc}' que matchea sin ambiguedad en desc_fundar."
                                     })
                else:
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y tiene la string '{norm_desc}' que NO matchea sin ambiguedad en desc_fundar. Se debe generar un código nuevo (a revisión para ser desambiguado con posibles matches)."
                                    })
                    
            # A.2) no hay columna desc_fundar para intentar
            else:
                problems.append({
                    'TOPICO': TOPICO,
                    'csv': ARCHIVO,
                    'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y no existe columna desc_fundar. A revisar qué se hace." 
                                })

    # Caso B: no existe columna de geocódigo, pero si se tiene de desc_fundar
    elif pd.notna(col_desc):
        raw_desc = str(df.at[idx, col_desc])
        norm_desc = sin_tildes(raw_desc.strip().upper())
        # busqueda coincidencias en desc_fundar
        matched_code = []
        for code_key, (_nl, desc0) in geonames_by_geocode.items():
            norm_desc0 = sin_tildes(str(desc0).strip().upper())
            if norm_desc0 == norm_desc:
                matched_code.append(code_key)
        if len(matched_code) == 1:
            c0 = matched_code[0]
            nl, desc0 = geonames_by_geocode[c0]
            df.at[idx, 'geocodigoFundar'] = c0
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
        else:
            problems.append({
                'TOPICO': TOPICO,
                'csv': ARCHIVO,
                'problema': f"Para la string '{norm_desc}' generar un codigo nuevo (a revison para ser desambiguado con posibles matches)"
                            })

    # Caso C: no hay columnas correspondientes a geocodigo ni de desc_fundar, entonces dejamos NaN

# gurdo csv modificado
os.makedirs(os.path.join(dest_dir, TOPICO), exist_ok = True)
dst_path = os.path.join(dest_dir, TOPICO, ARCHIVO)
df.to_csv(dst_path, index = False)
print(df)

# genero dataframe de problemas
df_problemas = pd.DataFrame(problems)
print(df_problemas)



     region_code                  region_name  year  poverty_line  \
0            AFE  Eastern and Southern Africa  1993          2.15   
1            AFE  Eastern and Southern Africa  1994          2.15   
2            AFE  Eastern and Southern Africa  1995          2.15   
3            AFE  Eastern and Southern Africa  1996          2.15   
4            AFE  Eastern and Southern Africa  1998          2.15   
...          ...                          ...   ...           ...   
1910         WLD                        World  2015         15.00   
1911         WLD                        World  2016         15.00   
1912         WLD                        World  2017         15.00   
1913         WLD                        World  2018         15.00   
1914         WLD                        World  2019         15.00   

      poverty_rate geocodigoFundar geonombreFundar  
0         0.584803            <NA>            <NA>  
1         0.589474            <NA>            <NA>  
2         0.

In [9]:
# PRUEBA SOBRE OTRO CSV DE UN TOPICO ARGENDATA CON PROBLEMA: 
# TOPICO    = 'PRECIO'  
# ARCHIVO   = '7_comparacion_inflacion_mediana_argentina_latam_1992_2022.csv'
# No se cumple condición X% Los valores de esta columna son: 'Argentina', 'América Latina y Caribe' 
# 'América Latina y Caribe' no aparece en el geonomenclador, si aparece ‘América Latina y el Caribe’. 
# No se lee columna similar ni a geocodigo ni a desc_fundar

import os
import pandas as pd
import unicodedata

# funcion eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in nkfd if not unicodedata.combining(c))

# parametros: especificar el tópico y el csv de ese topico
TOPICO    = 'PRECIO'  
ARCHIVO   = '7_comparacion_inflacion_mediana_argentina_latam_1992_2022.csv'

# rutas
base_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata'
dest_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata_new'

# df inputs
df_cod = df_codificacion
df_geo = df_geonomenclador
df_cols = columnscsv_Geocodigo_descFundar  

# dicc de name_long y desc_fundar por geocodigo del geonomenclador argendata
geonames_by_geocode = {}
for _, row in df_geo.iterrows():
    code = sin_tildes(str(row['geocodigo']).strip().upper())
    geonames_by_geocode[code] = (row['name_long'], row['desc_fundar'])

# abro el csv con respectivos encoding y delimiter, cfg de configuracion 
cfg = df_cod[(df_cod['TOPICO']==TOPICO) & 
             (df_cod['archivo_csv']==ARCHIVO)]
if cfg.empty:
    print(f'No se encontro info de encoding/delimiter para {TOPICO}/{ARCHIVO}')
encoding   = cfg.iloc[0]['encoding']
delimiter = cfg.iloc[0]['delimiter']

src_path = os.path.join(base_dir, TOPICO, ARCHIVO)
df = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) #csv de interes como un dartaframe
df_original = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) 

# print(df)

# le sumo al csv las nuevas columnas de interes
df['geocodigoFundar'] = pd.NA
df['geonombreFundar'] = pd.NA

print(df)

# del df columnscsv_Geocodigo_descFundar toma la info del csv y topico que me interesa
row_cols = df_cols[(df_cols['TOPICO'] == TOPICO) & (df_cols['archivo_csv'] == ARCHIVO)]
col_code = row_cols.iloc[0]['columna_Geocodigo']   if not row_cols.empty else None
col_desc = row_cols.iloc[0]['columna_DescFundar']  if not row_cols.empty else None

print(col_code)

# lista para tomar nota de los "problemas"
problems = []

# recorro cada fila del 
for idx in df.index:
    # CASO A: existe columna correspondiente a geocodigo del geonomenclador en el csv

    # A.0) hay un codigo presente en geocodigos, lo relleno con name_long o desc_fundar
    if pd.notna(col_code):
        raw_code = str(df.at[idx, col_code])
        norm_code = sin_tildes(raw_code.strip().upper())
        if norm_code in geonames_by_geocode:
            nl, desc0 = geonames_by_geocode[norm_code]
            df.at[idx, 'geocodigoFundar'] = norm_code
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0 # relleno con name_long si existe, sino relleno con la sting de desc_fundar
        else:
            # A.1) hay un codigo NO presente en geocodigos, pero intento por columna desc_fundar
            if pd.notna(col_desc):
                raw_desc = str(df.at[idx, col_desc])
                norm_desc = sin_tildes(raw_desc.strip().upper()) # normalizo columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%
                # bsqueda de coincidencias en desc_fundar
                matched_code = []
                for code_key, (_nl, desc0) in geonames_by_geocode.items(): # _nl para hacer explicito que no uso nl en este caso
                    norm_desc0 = sin_tildes(str(desc0).strip().upper()) # normalizo desc_fundar del geonomenclador 
                    #(para comparacion con la columna del csv de interes columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%)
                    if norm_desc0 == norm_desc:
                        matched_code.append(code_key)
                if len(matched_code) == 1:
                    c0 = matched_code[0]
                    nl, desc0 = geonames_by_geocode[c0]
                    df.at[idx, 'geocodigoFundar'] = c0
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}', que NO esta incluido en los geocodigos pero tiene la string '{norm_desc}' que matchea sin ambiguedad en desc_fundar."
                                     })
                else:
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y tiene la string '{norm_desc}' que NO matchea sin ambiguedad en desc_fundar. Se debe generar un código nuevo (a revisión para ser desambiguado con posibles matches)."
                                    })
                    
            # A.2) no hay columna desc_fundar para intentar
            else:
                problems.append({
                    'TOPICO': TOPICO,
                    'csv': ARCHIVO,
                    'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y no existe columna desc_fundar. A revisar qué se hace." 
                                })

    # Caso B: no existe columna de geocódigo, pero si se tiene de desc_fundar
    elif pd.notna(col_desc):
        raw_desc = str(df.at[idx, col_desc])
        norm_desc = sin_tildes(raw_desc.strip().upper())
        # busqueda coincidencias en desc_fundar
        matched_code = []
        for code_key, (_nl, desc0) in geonames_by_geocode.items():
            norm_desc0 = sin_tildes(str(desc0).strip().upper())
            if norm_desc0 == norm_desc:
                matched_code.append(code_key)
        if len(matched_code) == 1:
            c0 = matched_code[0]
            nl, desc0 = geonames_by_geocode[c0]
            df.at[idx, 'geocodigoFundar'] = c0
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
        else:
            problems.append({
                'TOPICO': TOPICO,
                'csv': ARCHIVO,
                'problema': f"Para la string '{norm_desc}' generar un codigo nuevo (a revison para ser desambiguado con posibles matches)"
                            })

    # Caso C: no hay columnas correspondientes a geocodigo ni de desc_fundar, entonces dejamos NaN

# gurdo csv modificado
os.makedirs(os.path.join(dest_dir, TOPICO), exist_ok = True)
dst_path = os.path.join(dest_dir, TOPICO, ARCHIVO)
df.to_csv(dst_path, index = False)
print(df)

# genero dataframe de problemas
df_problemas = pd.DataFrame(problems)
print(df_problemas)



    anio  mediana_paises_inflacion   pais_o_grupo_de_paises geocodigoFundar  \
0   1992                 17.545874                Argentina            <NA>   
1   1993                  7.364981                Argentina            <NA>   
2   1994                  3.854381                Argentina            <NA>   
3   1995                  1.607754                Argentina            <NA>   
4   1996                  0.054311                Argentina            <NA>   
..   ...                       ...                      ...             ...   
59  2019                  3.531500  América Latina y Caribe            <NA>   
60  2020                  3.040500  América Latina y Caribe            <NA>   
61  2021                  6.982000  América Latina y Caribe            <NA>   
62  2022                  8.375500  América Latina y Caribe            <NA>   
63  2023                  4.299000  América Latina y Caribe            <NA>   

   geonombreFundar  
0             <NA>  
1        

In [11]:
# PRUEBA SOBRE OTRO CSV DE UN TOPICO ARGENDATA CON PROBLEMA: 
# TOPICO    = 'TRANEN'  
# ARCHIVO   = 'potencia_instalada_renov_regional.csv'
# No se cumple condición X% Los valores de esta columna son: 'CABA y Provincia de Buenos Aires', 'Centro', 'Comahue',
# 'Cuyo', 'Litoral', 'Noreste Argentino', 'Noroeste Argentino', 'Patagonia', 'Total'
# 'CABA y Provincia de Buenos Aires',  'Noreste Argentino', 'Noroeste Argentino' no aparece en el geonomenclador.
# No se lee columna similar ni a geocodigo ni a desc_fundar

import os
import pandas as pd
import unicodedata

# funcion eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return ''.join(c for c in nkfd if not unicodedata.combining(c))

# parametros: especificar el tópico y el csv de ese topico
TOPICO    = 'TRANEN'  
ARCHIVO   = 'potencia_instalada_renov_regional.csv'

# rutas
base_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata'
dest_dir = '/home/capuccino/Desktop/TrabajoFundar/Argendata_renaming/data/data_argendata_new'

# df inputs
df_cod = df_codificacion
df_geo = df_geonomenclador
df_cols = columnscsv_Geocodigo_descFundar  

# dicc de name_long y desc_fundar por geocodigo del geonomenclador argendata
geonames_by_geocode = {}
for _, row in df_geo.iterrows():
    code = sin_tildes(str(row['geocodigo']).strip().upper())
    geonames_by_geocode[code] = (row['name_long'], row['desc_fundar'])

# abro el csv con respectivos encoding y delimiter, cfg de configuracion 
cfg = df_cod[(df_cod['TOPICO']==TOPICO) & 
             (df_cod['archivo_csv']==ARCHIVO)]
if cfg.empty:
    print(f'No se encontro info de encoding/delimiter para {TOPICO}/{ARCHIVO}')
encoding   = cfg.iloc[0]['encoding']
delimiter = cfg.iloc[0]['delimiter']

src_path = os.path.join(base_dir, TOPICO, ARCHIVO)
df = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) #csv de interes como un dartaframe
df_original = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter) 

# print(df)

# le sumo al csv las nuevas columnas de interes
df['geocodigoFundar'] = pd.NA
df['geonombreFundar'] = pd.NA

print(df)

# del df columnscsv_Geocodigo_descFundar toma la info del csv y topico que me interesa
row_cols = df_cols[(df_cols['TOPICO'] == TOPICO) & (df_cols['archivo_csv'] == ARCHIVO)]
col_code = row_cols.iloc[0]['columna_Geocodigo']   if not row_cols.empty else None
col_desc = row_cols.iloc[0]['columna_DescFundar']  if not row_cols.empty else None

print(col_code)

# lista para tomar nota de los "problemas"
problems = []

# recorro cada fila del 
for idx in df.index:
    # CASO A: existe columna correspondiente a geocodigo del geonomenclador en el csv

    # A.0) hay un codigo presente en geocodigos, lo relleno con name_long o desc_fundar
    if pd.notna(col_code):
        raw_code = str(df.at[idx, col_code])
        norm_code = sin_tildes(raw_code.strip().upper())
        if norm_code in geonames_by_geocode:
            nl, desc0 = geonames_by_geocode[norm_code]
            df.at[idx, 'geocodigoFundar'] = norm_code
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0 # relleno con name_long si existe, sino relleno con la sting de desc_fundar
        else:
            # A.1) hay un codigo NO presente en geocodigos, pero intento por columna desc_fundar
            if pd.notna(col_desc):
                raw_desc = str(df.at[idx, col_desc])
                norm_desc = sin_tildes(raw_desc.strip().upper()) # normalizo columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%
                # bsqueda de coincidencias en desc_fundar
                matched_code = []
                for code_key, (_nl, desc0) in geonames_by_geocode.items(): # _nl para hacer explicito que no uso nl en este caso
                    norm_desc0 = sin_tildes(str(desc0).strip().upper()) # normalizo desc_fundar del geonomenclador 
                    #(para comparacion con la columna del csv de interes columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%)
                    if norm_desc0 == norm_desc:
                        matched_code.append(code_key)
                if len(matched_code) == 1:
                    c0 = matched_code[0]
                    nl, desc0 = geonames_by_geocode[c0]
                    df.at[idx, 'geocodigoFundar'] = c0
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}', que NO esta incluido en los geocodigos pero tiene la string '{norm_desc}' que matchea sin ambiguedad en desc_fundar."
                                     })
                else:
                    problems.append({
                        'TOPICO': TOPICO,
                        'csv': ARCHIVO,
                        'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y tiene la string '{norm_desc}' que NO matchea sin ambiguedad en desc_fundar. Se debe generar un código nuevo (a revisión para ser desambiguado con posibles matches)."
                                    })
                    
            # A.2) no hay columna desc_fundar para intentar
            else:
                problems.append({
                    'TOPICO': TOPICO,
                    'csv': ARCHIVO,
                    'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y no existe columna desc_fundar. A revisar qué se hace." 
                                })

    # Caso B: no existe columna de geocódigo, pero si se tiene de desc_fundar
    elif pd.notna(col_desc):
        raw_desc = str(df.at[idx, col_desc])
        norm_desc = sin_tildes(raw_desc.strip().upper())
        # busqueda coincidencias en desc_fundar
        matched_code = []
        for code_key, (_nl, desc0) in geonames_by_geocode.items():
            norm_desc0 = sin_tildes(str(desc0).strip().upper())
            if norm_desc0 == norm_desc:
                matched_code.append(code_key)
        if len(matched_code) == 1:
            c0 = matched_code[0]
            nl, desc0 = geonames_by_geocode[c0]
            df.at[idx, 'geocodigoFundar'] = c0
            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
        else:
            problems.append({
                'TOPICO': TOPICO,
                'csv': ARCHIVO,
                'problema': f"Para la string '{norm_desc}' generar un codigo nuevo (a revison para ser desambiguado con posibles matches)"
                            })

    # Caso C: no hay columnas correspondientes a geocodigo ni de desc_fundar, entonces dejamos NaN

# gurdo csv modificado
os.makedirs(os.path.join(dest_dir, TOPICO), exist_ok = True)
dst_path = os.path.join(dest_dir, TOPICO, ARCHIVO)
df.to_csv(dst_path, index = False)
print(df)

# genero dataframe de problemas
df_problemas = pd.DataFrame(problems)
print(df_problemas)



                              region  tipo_energia  valor_en_mw  porcentaje  \
0   CABA y Provincia de Buenos Aires    Bioenergía           48    3.219316   
1   CABA y Provincia de Buenos Aires        Eólica         1443   96.780684   
2   CABA y Provincia de Buenos Aires  Fotovoltaica            0    0.000000   
3   CABA y Provincia de Buenos Aires         Hidro            0    0.000000   
4                             Centro    Bioenergía           34    6.679764   
5                             Centro        Eólica          240   47.151277   
6                             Centro  Fotovoltaica          118   23.182711   
7                             Centro         Hidro          117   22.986248   
8                            Comahue    Bioenergía            2    0.680272   
9                            Comahue        Eólica          253   86.054422   
10                           Comahue  Fotovoltaica            7    2.380952   
11                           Comahue         Hidro  

<b> <font color = "bisque"> LO CORRO PARA TODOS LOS TOPICOS </font> </b>

In [5]:
import os
import pandas as pd
import unicodedata

# funcion eliminar tildes
def sin_tildes(s):
    nkfd = unicodedata.normalize('NFKD', s)
    return "".join(c for c in nkfd if not unicodedata.combining(c))

# rutas
base_dir = '/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata'
dest_dir = '/home/lucia/Desktop/Fundar/Argendata_renaming/data/data_argendata_new'

# df inputs
df_cod = df_codificacion
df_geo = df_geonomenclador
df_cols = columnscsv_Geocodigo_descFundar  

# dicc de name_long y desc_fundar por geocodigo del geonomenclador argendata
geonames_by_geocode = {}
for _, row in df_geonomenclador.iterrows():
    raw_code   = str(row['geocodigo'])
    code_norm  = sin_tildes(raw_code.strip().upper())
    name_long  = row['name_long']
    desc_fund  = row['desc_fundar']
    geonames_by_geocode[code_norm] = (name_long, desc_fund)

# lista problemas por tipo
problemas = []

# lista de topicos (en este caso es solo pesca)
topics = []
for d in os.listdir(base_dir):
    path = os.path.join(base_dir, d)
    if os.path.isdir(path) and d not in ('LICENSE', 'README.md'):
        topics.append(d)

# proceso cada topico y csv
for topico in topics:
    src_topic = os.path.join(base_dir, topico)
    dst_topic = os.path.join(dest_dir, topico)
    os.makedirs(dst_topic, exist_ok = True)
    
    # listar los archivos csv en este tópico
    for archivo in os.listdir(src_topic):
        if not archivo.lower().endswith('.csv'):
            continue
        
        # abro el csv con respectivos encoding y delimiter, cfg de configuracion 
        cfg = df_codificacion[
            (df_codificacion['TOPICO'] == topico) &
            (df_codificacion['archivo_csv'] == archivo)
        ]
        if cfg.empty:
            print(f"No hay info encoding/delimiter para {topico}/{archivo}")
            continue
        encoding = cfg.iloc[0]['encoding']
        delimiter = cfg.iloc[0]['delimiter']
        
        # del df columnscsv_Geocodigo_descFundar toma la info del csv y topico que me interesa
        row_cols = columnscsv_Geocodigo_descFundar[
            (columnscsv_Geocodigo_descFundar['TOPICO'] == topico) &
            (columnscsv_Geocodigo_descFundar['archivo_csv'] == archivo)
        ]
        col_code = row_cols.iloc[0]['columna_Geocodigo'] if not row_cols.empty else None
        col_desc = row_cols.iloc[0]['columna_DescFundar'] if not row_cols.empty else None
        
        # leo csv
        src_path = os.path.join(src_topic, archivo)
        df = pd.read_csv(src_path, encoding = encoding, delimiter = delimiter)
        
        # le sumo al csv las nuevas columnas de interes
        df['geocodigoFundar'] = pd.NA
        df['geonombreFundar'] = pd.NA
        
        # recorrer filas del csv
        for idx in df.index:

            # CASO A: existe columna correspondiente a geocodigo del geonomenclador en el csv

            # A.0) hay un codigo presente en geocodigos, lo relleno con name_long o desc_fundar
            
            if pd.notna(col_code):
                raw_code = str(df.at[idx, col_code])
                norm_code = sin_tildes(raw_code.strip().upper())
                if norm_code in geonames_by_geocode:
                    nl, desc0 = geonames_by_geocode[norm_code]
                    df.at[idx, 'geocodigoFundar'] = norm_code
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0 # relleno con name_long si existe, sino relleno con la sting de desc_fundar
                else:
                    # A.1) hay un codigo NO presente en geocodigos, pero intento por columna desc_fundar
                    if pd.notna(col_desc):
                        raw_desc = str(df.at[idx, col_desc])
                        norm_desc = sin_tildes(raw_desc.strip().upper()) # normalizo columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%
                        # bsqueda de coincidencias en desc_fundar
                        matched_code = []
                        for code_key, (_nl, desc0) in geonames_by_geocode.items(): # _nl para hacer explicito que no uso nl en este caso
                            norm_desc0 = sin_tildes(str(desc0).strip().upper()) # normalizo desc_fundar del geonomenclador 
                            #(para comparacion con la columna del csv de interes columna del csv de interes que se corresponderia a desc_fundar segun condicion del X%)
                            if norm_desc0 == norm_desc:
                                matched_code.append(code_key)
                        if len(matched_code) == 1:
                            c0 = matched_code[0]
                            nl, desc0 = geonames_by_geocode[c0]
                            df.at[idx, 'geocodigoFundar'] = c0
                            df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                            problemas.append({
                                'TOPICO': topico,
                                'csv': archivo,
                                'problema': f"dataset tiene el codigo '{norm_code}', que NO esta incluido en los geocodigos pero tiene la string '{norm_desc}' que matchea sin ambiguedad en desc_fundar. No hay que modificar geonomenclador."
                            })
                        else:
                            problemas.append({
                                'TOPICO': topico,
                                'csv': archivo,
                                'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y tiene la string '{norm_desc}' que NO matchea sin ambiguedad en desc_fundar. Puede implicar modificaciones del geonomenclador (a revisión, para no pisarse con otros casos)."
                            })
                    # A.2) no hay columna desc_fundar para intentar

                    else:
                        problemas.append({
                            'TOPICO': topico,
                            'csv': archivo,
                            'problema': f"dataset tiene el codigo '{norm_code}' que NO esta incluido en los geocodigos y no existe columna desc_fundar. Puede implicar modificaciones del geonomenclador (a revisión, para no pisarse con otros casos)."
                        })
            
            # CASO B: no existe columna de geocodigo, pero si se tiene de desc_fundar

            elif pd.notna(col_desc):
                raw_desc = str(df.at[idx, col_desc])
                norm_desc = sin_tildes(raw_desc.strip().upper())
                matched_code = []
                for code_key, (_nl, desc0) in geonames_by_geocode.items():
                    if sin_tildes(str(desc0).strip().upper()) == norm_desc:
                        matched_code.append(code_key)
                if len(matched_code) == 1:
                    c0 = matched_code[0]
                    nl, desc0 = geonames_by_geocode[c0]
                    df.at[idx, 'geocodigoFundar'] = c0
                    df.at[idx, 'geonombreFundar'] = nl if pd.notna(nl) else desc0
                else:
                    problemas.append({
                        'TOPICO': topico,
                        'csv': archivo,
                        'problema': f"Para la string '{norm_desc}' se debería generar un codigo nuevo. Puede implicar modificaciones del geonomenclador (a revisión, para no pisarse con otros casos)."
                    })
            
            # CASO C: si no hay columnas asociadas, quedan NaN
        
        # guardar csv copia modificado en su correspondiente topico, con UTF-8 y separador coma
        dst_path = os.path.join(dest_dir, topico, archivo)
        df.to_csv(dst_path, index = False,  encoding = 'utf-8', sep = ',')

# df de problemas
df_problemas = pd.DataFrame(problemas) # eliminar duplicados de "problemas" que se repiten para TOPICO | csv
df_problemas = df_problemas.drop_duplicates(subset=['TOPICO', 'csv', 'problema'])
df_problemas.to_csv('/home/lucia/Desktop/Fundar/Argendata_renaming/data/df_problemas.csv', index = False,  encoding = 'utf-8', sep = ',')
print(df_problemas)


       TOPICO                                                csv  \
0      SEBACO                      22_participacion_expo_sbc.csv   
18     SEBACO                      22_participacion_expo_sbc.csv   
36     SEBACO                      23_participacion_expo_ssi.csv   
54     SEBACO                        06_empleo_sbc_provincia.csv   
81     SEBACO                        06_empleo_sbc_provincia.csv   
...       ...                                                ...   
33406  ACECON                      A5_pibpc_propeeuu_ppa_usd.csv   
33449  INFDES  tasa_informalidad_provincia_definicion_product...   
33465  INFDES  tasa_informalidad_provincia_definicion_product...   
33481  MINERI             exportaciones_mineras_provinciales.csv   
33531  MINERI             exportaciones_mineras_provinciales.csv   

                                                problema  
0      dataset tiene el codigo 'CHT' que NO esta incl...  
18     dataset tiene el codigo 'ROM' que NO esta incl...  
36    

<b> <font color = "bisque"> CLASIFICACION PROBLEMAS </font> </b>

In [6]:
# prueba de clasificacion problemas

# 1. codigo no en geocodigos pero desc_fundar coincide
mask1 = (
    df_problemas['problema'].str.contains(
        "que NO esta incluido en los geocodigos pero tiene la string"
    ) &
    df_problemas['problema'].str.contains("No hay que modificar geonomenclador")
)
df_tipo1 = df_problemas[mask1]
df_tipo1.to_csv('/home/lucia/Desktop/Fundar/Argendata_renaming/data/df_tipo1.csv', index = False,  encoding = 'utf-8', sep = ',')

# 2. codigo no en geocodigos y desc_fundar no coincide
mask2 = df_problemas['problema'].str.contains(
    "que NO esta incluido en los geocodigos y tiene la string"
) & df_problemas['problema'].str.contains(
    "que NO matchea sin ambiguedad en desc_fundar. Puede implicar modificaciones"
)
df_tipo2 = df_problemas[mask2]
df_tipo2.to_csv('/home/lucia/Desktop/Fundar/Argendata_renaming/data/df_tipo2.csv', index = False,  encoding = 'utf-8', sep = ',')

# 3. codigo no en geocodigos y no existe columna desc_fundar
mask3 = df_problemas['problema'].str.contains(
    "que NO esta incluido en los geocodigos y no existe columna desc_fundar"
)
df_tipo3 = df_problemas[mask3]
df_tipo3.to_csv('/home/lucia/Desktop/Fundar/Argendata_renaming/data/df_tipo3.csv', index = False,  encoding = 'utf-8', sep = ',')

# 4. Para la string ... se debería generar un codigo nuevo
mask4 = df_problemas['problema'].str.contains(
    "Para la string '.*' se debería generar un codigo nuevo",
    regex=True
)
df_tipo4 = df_problemas[mask4]
df_tipo4.to_csv('/home/lucia/Desktop/Fundar/Argendata_renaming/data/df_tipo4.csv', index = False,  encoding = 'utf-8', sep = ',')


In [16]:
df_tipo1

,TOPICO,csv,problema
58,ESTPRO,va_sectorial_pais.csv,"dataset tiene el codigo 'KOS', que NO esta inc..."
65,ESTPRO,va_sectorial_pais.csv,"dataset tiene el codigo 'SD1', que NO esta inc..."
72,ESTPRO,va_sectorial_pais.csv,"dataset tiene el codigo 'SV1', que NO esta inc..."
86,ESTPRO,va_sectorial_pais.csv,"dataset tiene el codigo 'TZ2', que NO esta inc..."
93,ESTPRO,va_sectorial_pais.csv,"dataset tiene el codigo 'YM1', que NO esta inc..."
...,...,...,...
33198,AGROPE,produccion_global_carne_bovina_evo.csv,"dataset tiene el codigo 'F51', que NO esta inc..."
33230,AGROPE,rindes_maiz_internacional.csv,"dataset tiene el codigo 'F15', que NO esta inc..."
33269,AGROPE,rindes_maiz_internacional.csv,"dataset tiene el codigo 'F228', que NO esta in..."
33300,AGROPE,rindes_maiz_internacional.csv,"dataset tiene el codigo 'F248', que NO esta in..."


In [17]:
df_tipo2

,TOPICO,csv,problema
79,ESTPRO,va_sectorial_pais.csv,dataset tiene el codigo 'TZ1' que NO esta incl...
2707,ESTPRO,particip_va_intensivos_id_bys.csv,dataset tiene el codigo 'EU28XEU15' que NO est...
2759,ESTPRO,particip_va_intensivos_id_bys.csv,dataset tiene el codigo 'NAFTA' que NO esta in...
2811,ESTPRO,particip_va_intensivos_id_bys.csv,dataset tiene el codigo 'S2_S8' que NO esta in...
2961,ESTPRO,va_sectorial_pais_bys.csv,dataset tiene el codigo 'TZ1' que NO esta incl...
...,...,...,...
32385,DESHUM,inb_pc.csv,dataset tiene el codigo 'ZZD.LHD' que NO esta ...
32418,DESHUM,inb_pc.csv,dataset tiene el codigo 'ZZE.AS' que NO esta i...
32550,DESHUM,inb_pc.csv,dataset tiene el codigo 'ZZI.SA' que NO esta i...
32583,DESHUM,inb_pc.csv,dataset tiene el codigo 'ZZJ.SSA' que NO esta ...


In [18]:
df_tipo3

,TOPICO,csv,problema
0,SALING,ISA_salarios_mundo_i1.csv,dataset tiene el codigo 'ELS' que NO esta incl...
1,SALING,ISA_salarios_mundo_i1.csv,dataset tiene el codigo 'PAR' que NO esta incl...
3780,SEBACO,22_participacion_expo_sbc.csv,dataset tiene el codigo 'CHT' que NO esta incl...
3798,SEBACO,22_participacion_expo_sbc.csv,dataset tiene el codigo 'ROM' que NO esta incl...
3816,SEBACO,23_participacion_expo_ssi.csv,dataset tiene el codigo 'ROM' que NO esta incl...
...,...,...,...
31153,DESHUM,idh_subcomponentes.csv,dataset tiene el codigo 'ZZG.ECA' que NO esta ...
31351,DESHUM,idh_subcomponentes.csv,dataset tiene el codigo 'ZZH.LAC' que NO esta ...
31549,DESHUM,idh_subcomponentes.csv,dataset tiene el codigo 'ZZI.SA' que NO esta i...
31747,DESHUM,idh_subcomponentes.csv,dataset tiene el codigo 'ZZJ.SSA' que NO esta ...


In [19]:
df_tipo4

,TOPICO,csv,problema
2,SALING,ISA_tipo_empleo_i3.csv,Para la string 'REP. DOMINICANA' se debería ge...
9,INFDES,tasa_informalidad_provincia_definicion_product...,Para la string 'CABA' se debería generar un co...
25,INFDES,tasa_informalidad_provincia_definicion_product...,Para la string 'TOTAL PAIS' se debería generar...
41,ESTPRO,densidad_empresarial_depto.csv,Para la string 'CIUDAD DE BUENOS AIRES' se deb...
55,ESTPRO,densidad_empresarial_depto.csv,Para la string 'NAN' se debería generar un cod...
2941,ESTPRO,densidad_nbi.csv,Para la string 'CIUDAD DE BUENOS AIRES' se deb...
2955,ESTPRO,densidad_nbi.csv,Para la string 'NAN' se debería generar un cod...
3461,ACECON,9_pibpc_ppa_log_1950.csv,Para la string 'TIGRES ASIATICOS' se debería g...
3726,SEBACO,06_empleo_sbc_provincia.csv,Para la string 'CABA' se debería generar un co...
3753,SEBACO,06_empleo_sbc_provincia.csv,Para la string 'GBA' se debería generar un cod...
